In [1]:
from pyspark.sql.types import StructField, StructType, StringType, LongType, FloatType, DateType, TimestampType, IntegerType
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql import Row
from pyspark.sql.functions import *
from pyspark.sql.types import *

import pyspark
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SQLContext

import matplotlib
import matplotlib.pyplot as plt

import atexit
from numpy import array
import numpy as np
import datetime

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1605985399375_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# 1. Dataset Set up

## 1.1 Import dataset

In [2]:
manualSchema = StructType([
    StructField('machine_id', LongType(), True),
    StructField('site_session_id', LongType(), True),
    StructField('prod_category_id', LongType(), True),
    StructField('prod_name', StringType(), True),
    StructField('domain_id', LongType(),True),
    StructField('prod_qty', LongType(), True),
    StructField('prod_totprice', FloatType(), True),
    StructField('basket_tot', LongType(), True),
    StructField('event_date', DateType(), True),
    StructField('event_time', TimestampType(), True),
    StructField('hoh_most_education', LongType(), True),
    StructField('census_region', LongType(), True),
    StructField('household_size', LongType(), True),
    StructField('hoh_oldest_age', LongType(), True),
    StructField('household_income', LongType(), True),
    StructField('children', LongType(), True),
    StructField('racial_background', LongType(), True),
    StructField('connection_speed', LongType(), True),
    StructField('country_of_origin', LongType(), True),
    StructField('zip_code', LongType(), True),
    StructField('domain_name', StringType(), True)
])

#transaction = spark.read.format('csv').schema(manualSchema).load("s3://jiayan-bigdata/Final Project/transaction 2019.csv")
transaction = spark.read.option("inferSchema", "true").option("header", "true").csv("s3://aws-logs-281175518142-us-east-1/Final_Project/transaction 2019.csv")
category = spark.read.option("inferSchema", "true").option("header","true").csv("s3://aws-logs-281175518142-us-east-1/Final_Project/Product_Categories.csv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 1.2 Take a look at their schema

In [3]:
# Check the schema of columns 
transaction.printSchema()

# Show the first 10 observations
transaction.show(10)

# Print the total number of rows
print("There are {} rows in the transaction DataFrame".format(transaction.count()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- machine_id: integer (nullable = true)
 |-- site_session_id: long (nullable = true)
 |-- prod_category_id: long (nullable = true)
 |-- prod_name: string (nullable = true)
 |-- domain_id: string (nullable = true)
 |-- prod_qty: string (nullable = true)
 |-- prod_totprice: string (nullable = true)
 |-- basket_tot: string (nullable = true)
 |-- event_date: string (nullable = true)
 |-- event_time: string (nullable = true)
 |-- hoh_most_education: string (nullable = true)
 |-- census_region: string (nullable = true)
 |-- household_size: string (nullable = true)
 |-- hoh_oldest_age: string (nullable = true)
 |-- household_income: string (nullable = true)
 |-- children: string (nullable = true)
 |-- racial_background: string (nullable = true)
 |-- connection_speed: string (nullable = true)
 |-- country_of_origin: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- domain_name: string (nullable = true)

+----------+-------------------+----------------+---------+----

In [4]:
# Check the schema of columns 
category.printSchema()

# Show the first 10 observations
category.show(10)

# Print the total number of rows
print("There are {} rows in the category DataFrame".format(category.count()))

# Print Schema
transaction.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Product Category ID: long (nullable = true)
 |-- Topline Category: string (nullable = true)
 |-- Report Category: string (nullable = true)
 |-- Report Sub-Category: string (nullable = true)
 |-- Item Category: string (nullable = true)
 |-- Item Sub-Category: string (nullable = true)

+-------------------+----------------+--------------------+-------------------+------------------+------------------+
|Product Category ID|Topline Category|     Report Category|Report Sub-Category|     Item Category| Item Sub-Category|
+-------------------+----------------+--------------------+-------------------+------------------+------------------+
|                 43|          TRAVEL|          AIR TRAVEL|         AIR TRAVEL|        AIR TRAVEL|        AIR TRAVEL|
|                 44|          TRAVEL|  HOTEL RESERVATIONS| HOTEL RESERVATIONS|HOTEL RESERVATIONS|HOTEL RESERVATIONS|
|                 45|          TRAVEL|          CAR RENTAL|         CAR RENTAL|        CAR RENTAL|        CAR RENTA

In [5]:
transaction = transaction.withColumn("prod_qty",col("prod_qty").cast(IntegerType()))
transaction = transaction.withColumn("prod_category_id",col("prod_category_id").cast(IntegerType()))
transaction = transaction.withColumn("domain_id",col("domain_id").cast(IntegerType()))
transaction = transaction.withColumn("prod_totprice",col("prod_totprice").cast(IntegerType()))
transaction = transaction.withColumn("prod_qty",col("prod_qty").cast("float"))
transaction = transaction.withColumn("prod_totprice",col("prod_totprice").cast("float"))
transaction = transaction.withColumn("basket_tot",col("basket_tot").cast("float"))

# Check the schema again
transaction.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- machine_id: integer (nullable = true)
 |-- site_session_id: long (nullable = true)
 |-- prod_category_id: integer (nullable = true)
 |-- prod_name: string (nullable = true)
 |-- domain_id: integer (nullable = true)
 |-- prod_qty: float (nullable = true)
 |-- prod_totprice: float (nullable = true)
 |-- basket_tot: float (nullable = true)
 |-- event_date: string (nullable = true)
 |-- event_time: string (nullable = true)
 |-- hoh_most_education: string (nullable = true)
 |-- census_region: string (nullable = true)
 |-- household_size: string (nullable = true)
 |-- hoh_oldest_age: string (nullable = true)
 |-- household_income: string (nullable = true)
 |-- children: string (nullable = true)
 |-- racial_background: string (nullable = true)
 |-- connection_speed: string (nullable = true)
 |-- country_of_origin: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- domain_name: string (nullable = true)

In [6]:
transaction.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+----------------+---------+---------+--------+-------------+----------+----------+----------+------------------+-------------+--------------+--------------+----------------+--------+-----------------+----------------+-----------------+--------+-----------+
|machine_id|    site_session_id|prod_category_id|prod_name|domain_id|prod_qty|prod_totprice|basket_tot|event_date|event_time|hoh_most_education|census_region|household_size|hoh_oldest_age|household_income|children|racial_background|connection_speed|country_of_origin|zip_code|domain_name|
+----------+-------------------+----------------+---------+---------+--------+-------------+----------+----------+----------+------------------+-------------+--------------+--------------+----------------+--------+-----------------+----------------+-----------------+--------+-----------+
| 182325451|6842979938964571944|              43|     null|     null|     1.0|         null|     251.0|  20190722|   2:37:18|        

# 2. Exploratory Data Analysis

In [7]:
# We are only interested in the book categories - let's filter
transaction_filtered = transaction.filter((transaction.prod_category_id==2003005058)|(transaction.prod_category_id==2003005059)|(transaction.prod_category_id==2003005060)|(transaction.prod_category_id==2003005061)|(transaction.prod_category_id==2003005062)|(transaction.prod_category_id==2003005063)|(transaction.prod_category_id==2003005064)|(transaction.prod_category_id==2003005065)|(transaction.prod_category_id==2003005066)|(transaction.prod_category_id==2003005067)|(transaction.prod_category_id==2003005068)|(transaction.prod_category_id==2003005069)|(transaction.prod_category_id==2003005070)|(transaction.prod_category_id==2003005071)|(transaction.prod_category_id==2003005072)|(transaction.prod_category_id==2003005073)|(transaction.prod_category_id==2003005074))
transaction_filtered.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+----------------+--------------------+---------+--------+-------------+----------+----------+----------+------------------+-------------+--------------+--------------+----------------+--------+-----------------+----------------+-----------------+--------+-----------+
|machine_id|    site_session_id|prod_category_id|           prod_name|domain_id|prod_qty|prod_totprice|basket_tot|event_date|event_time|hoh_most_education|census_region|household_size|hoh_oldest_age|household_income|children|racial_background|connection_speed|country_of_origin|zip_code|domain_name|
+----------+-------------------+----------------+--------------------+---------+--------+-------------+----------+----------+----------+------------------+-------------+--------------+--------------+----------------+--------+-----------------+----------------+-----------------+--------+-----------+
| 212148765|6660154091378260634|      2003005073|Spelling & Poetry...|     null|     1.0|          5

In [8]:
# We are only interested in the book categories that are sold in Amazon.com
transaction_filtered = transaction_filtered.filter((transaction_filtered.domain_name=="amazon.com"))
transaction_filtered.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+----------------+--------------------+---------+--------+-------------+----------+----------+----------+------------------+-------------+--------------+--------------+----------------+--------+-----------------+----------------+-----------------+--------+-----------+
|machine_id|    site_session_id|prod_category_id|           prod_name|domain_id|prod_qty|prod_totprice|basket_tot|event_date|event_time|hoh_most_education|census_region|household_size|hoh_oldest_age|household_income|children|racial_background|connection_speed|country_of_origin|zip_code|domain_name|
+----------+-------------------+----------------+--------------------+---------+--------+-------------+----------+----------+----------+------------------+-------------+--------------+--------------+----------------+--------+-----------------+----------------+-----------------+--------+-----------+
| 171914940|5412109291078152593|      2003005073|1st Grade Spellin...|     null|     1.0|          9

In [9]:
# We are only selecting the columns we will need for the analysis
transaction_filtered_selected = transaction_filtered.select('machine_id','prod_category_id','prod_name','prod_qty','prod_totprice','basket_tot','hoh_most_education','census_region','household_size','hoh_oldest_age','household_income','children','children','connection_speed','country_of_origin','zip_code')
transaction_filtered_selected.toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

       machine_id  prod_category_id  ... country_of_origin  zip_code
0       171914940        2003005073  ...                 0     29405
1       171914940        2003005063  ...                 0     29405
2       171914940        2003005063  ...                 0     29405
3       171914940        2003005063  ...                 0     29405
4       171914940        2003005063  ...                 0     29405
...           ...               ...  ...               ...       ...
18878   284909864        2003005060  ...                 0     32081
18879   284909864        2003005063  ...                 0     32081
18880   284910736        2003005073  ...                 1     08902
18881   284910736        2003005073  ...                 1     08902
18882   284912202        2003005060  ...                 0      None

[18883 rows x 16 columns]

In [10]:
# Update the value inside the 'prod_category_id' column to the respective book sub-category value
transaction_mapped1 = transaction_filtered_selected.withColumn('prod_category_id', regexp_replace('prod_category_id', '2003005058', 'ART, MUSIC & PHOTOGRAPHY'))
transaction_mapped2 = transaction_mapped1.withColumn('prod_category_id', regexp_replace('prod_category_id', '2003005059', 'BIOGRAPHY & MEMOIRS'))
transaction_mapped3 = transaction_mapped2.withColumn('prod_category_id', regexp_replace('prod_category_id', '2003005060', 'GENERAL BOOKS'))
transaction_mapped4 = transaction_mapped3.withColumn('prod_category_id', regexp_replace('prod_category_id', '2003005061', 'BUSINESS & INVESTING'))
transaction_mapped5 = transaction_mapped4.withColumn('prod_category_id', regexp_replace('prod_category_id', '2003005062', 'CALENDARS'))
transaction_mapped6 = transaction_mapped5.withColumn('prod_category_id', regexp_replace('prod_category_id', '2003005063', 'CHILDREN BOOKS'))
transaction_mapped7 = transaction_mapped6.withColumn('prod_category_id', regexp_replace('prod_category_id', '2003005064', 'COOKING, FOOD & WINE'))
transaction_mapped8 = transaction_mapped7.withColumn('prod_category_id', regexp_replace('prod_category_id', '2003005065', 'HEALTH, MIND & BODY'))
transaction_mapped9 = transaction_mapped8.withColumn('prod_category_id', regexp_replace('prod_category_id', '2003005066', 'HISTORY'))
transaction_mapped10 = transaction_mapped9.withColumn('prod_category_id', regexp_replace('prod_category_id', '2003005067', 'HOME, HOBBIES & GARDEN'))
transaction_mapped11 = transaction_mapped10.withColumn('prod_category_id', regexp_replace('prod_category_id', '2003005068', 'LITERATURE & FICTION'))
transaction_mapped12 = transaction_mapped11.withColumn('prod_category_id', regexp_replace('prod_category_id', '2003005069', 'PARENTING & FAMILIES'))
transaction_mapped13 = transaction_mapped12.withColumn('prod_category_id', regexp_replace('prod_category_id', '2003005070', 'REFERENCE'))
transaction_mapped14 = transaction_mapped13.withColumn('prod_category_id', regexp_replace('prod_category_id', '2003005071', 'RELIGION'))
transaction_mapped15 = transaction_mapped14.withColumn('prod_category_id', regexp_replace('prod_category_id', '2003005072', 'ROMANCE'))
transaction_mapped16 = transaction_mapped15.withColumn('prod_category_id', regexp_replace('prod_category_id', '2003005073', 'TEXTBOOKS'))
transaction_mapped = transaction_mapped16.withColumn('prod_category_id', regexp_replace('prod_category_id', '2003005074', 'VIDEO GAMES STRATEGY GUIDES'))
transaction_mapped.limit(10).toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   machine_id prod_category_id  ... country_of_origin  zip_code
0   171914940        TEXTBOOKS  ...                 0     29405
1   171914940   CHILDREN BOOKS  ...                 0     29405
2   171914940   CHILDREN BOOKS  ...                 0     29405
3   171914940   CHILDREN BOOKS  ...                 0     29405
4   171914940   CHILDREN BOOKS  ...                 0     29405
5   179376772    GENERAL BOOKS  ...                 0     01057
6   179376772    GENERAL BOOKS  ...                 0     01057
7   192415223   CHILDREN BOOKS  ...                 0     78703
8   205955364        TEXTBOOKS  ...                 0     92084
9   205955364        TEXTBOOKS  ...                 0     92084

[10 rows x 16 columns]

In [11]:
transaction_mapped.groupBy(["prod_category_id"]).agg(F.sum("prod_qty")).show()
transaction_mapped.groupBy(["prod_category_id"]).agg(F.sum("basket_tot")).show()
transaction_mapped.groupBy(["prod_category_id"]).agg(F.sum("prod_totprice")).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------+
|    prod_category_id|sum(prod_qty)|
+--------------------+-------------+
|             ROMANCE|          3.0|
|ART, MUSIC & PHOT...|       1330.0|
|           REFERENCE|          1.0|
|HOME, HOBBIES & G...|          1.0|
|           CALENDARS|          9.0|
| HEALTH, MIND & BODY|          7.0|
|LITERATURE & FICTION|       2232.0|
|            RELIGION|          5.0|
| BIOGRAPHY & MEMOIRS|          1.0|
|COOKING, FOOD & WINE|          2.0|
|           TEXTBOOKS|       2640.0|
|       GENERAL BOOKS|       6764.0|
|      CHILDREN BOOKS|       7106.0|
+--------------------+-------------+

+--------------------+------------------+
|    prod_category_id|   sum(basket_tot)|
+--------------------+------------------+
|             ROMANCE| 4.980000019073486|
|ART, MUSIC & PHOT...| 43358.18020552397|
|           REFERENCE|24.399999618530273|
|HOME, HOBBIES & G...| 7.949999809265137|
|           CALENDARS|   493.01001060009|
| HEALTH, MIND & BODY| 1242.129970

In [12]:
transaction_mapped.select("prod_category_id","prod_qty").groupBy("prod_category_id").agg(avg("prod_qty")).show()
transaction_mapped.select("prod_category_id","basket_tot").groupBy("prod_category_id").agg(avg("basket_tot")).show()
transaction_mapped.select("prod_category_id","prod_totprice").groupBy("prod_category_id").agg(avg("prod_totprice")).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+
|    prod_category_id|     avg(prod_qty)|
+--------------------+------------------+
|             ROMANCE|               1.0|
|ART, MUSIC & PHOT...|1.0350194552529184|
|           REFERENCE|               1.0|
|HOME, HOBBIES & G...|               1.0|
|           CALENDARS|               1.0|
| HEALTH, MIND & BODY|1.1666666666666667|
|LITERATURE & FICTION|1.0072202166064983|
|            RELIGION|               1.0|
| BIOGRAPHY & MEMOIRS|               1.0|
|COOKING, FOOD & WINE|               1.0|
|           TEXTBOOKS|1.0501193317422435|
|       GENERAL BOOKS|1.0570401625253947|
|      CHILDREN BOOKS| 1.103244837758112|
+--------------------+------------------+

+--------------------+------------------+
|    prod_category_id|   avg(basket_tot)|
+--------------------+------------------+
|             ROMANCE|1.6600000063578289|
|ART, MUSIC & PHOT...| 33.74177447900698|
|           REFERENCE|24.399999618530273|
|HOME, HOBBIES & G...| 7.94999980

In [13]:
transaction_filtered.registerTempTable('transaction_table')
newDF = sqlContext.sql('select * from transaction_table where basket_tot>1000')
newDF.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+----------------+--------------------+---------+--------+-------------+----------+----------+----------+------------------+-------------+--------------+--------------+----------------+--------+-----------------+----------------+-----------------+--------+-----------+
|machine_id|    site_session_id|prod_category_id|           prod_name|domain_id|prod_qty|prod_totprice|basket_tot|event_date|event_time|hoh_most_education|census_region|household_size|hoh_oldest_age|household_income|children|racial_background|connection_speed|country_of_origin|zip_code|domain_name|
+----------+-------------------+----------------+--------------------+---------+--------+-------------+----------+----------+----------+------------------+-------------+--------------+--------------+----------------+--------+-----------------+----------------+-----------------+--------+-----------+
| 259228973|4905657229081595999|      2003005060|State Quarter 199...|     null|     1.0|          4

In [14]:
windowSpec = Window().partitionBy(['prod_category_id']).orderBy(F.desc('prod_qty'))
transaction_filtered.withColumn("rank",F.rank().over(windowSpec)).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+----------------+--------------------+---------+--------+-------------+----------+----------+----------+------------------+-------------+--------------+--------------+----------------+--------+-----------------+----------------+-----------------+--------+-----------+----+
|machine_id|    site_session_id|prod_category_id|           prod_name|domain_id|prod_qty|prod_totprice|basket_tot|event_date|event_time|hoh_most_education|census_region|household_size|hoh_oldest_age|household_income|children|racial_background|connection_speed|country_of_origin|zip_code|domain_name|rank|
+----------+-------------------+----------------+--------------------+---------+--------+-------------+----------+----------+----------+------------------+-------------+--------------+--------------+----------------+--------+-----------------+----------------+-----------------+--------+-----------+----+
| 271296111|3793700835024179591|      2003005062|Bass 2019 Wall Ca...|     null|     

# SQL Queries

# Data Visualization